In [47]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import pandas as pd
from pandas_datareader import data as pdr
import datetime
import pickle as pkl



In [14]:
start = datetime.datetime(2005, 1, 1)
end = datetime.datetime(2024, 12, 31)

In [15]:
features = {'Inflation': 'CPIAUCSL', 'wages': 'FRBATLWGTUMHWGO', 'unemployment': 'UNRATE'}


In [16]:
df = pd.DataFrame(pdr.DataReader('CSUSHPISA', 'fred', start, end)).rename(columns={'CSUSHPISA': 'home_price'})

for key, value in features.items():
    temp = pd.DataFrame(pdr.DataReader(f'{value}', 'fred', start, end)).rename(columns={'value': 'key'})
    df = pd.concat([df, temp], axis=1)


In [17]:
df

,home_price,CPIAUCSL,FRBATLWGTUMHWGO,UNRATE
DATE,,,,
2005-01-01,161.289,191.600,3.9,5.3
2005-02-01,163.346,192.400,3.4,5.4
2005-03-01,165.814,193.100,3.4,5.2
2005-04-01,167.503,193.700,3.7,5.2
2005-05-01,169.352,193.600,3.8,5.1
...,...,...,...,...
2024-08-01,322.282,314.131,4.7,4.2
2024-09-01,323.282,314.851,4.9,4.1
2024-10-01,324.466,315.564,4.2,4.1


In [19]:
X = df.drop(['home_price'], axis=1)
y = df['home_price']
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
X_train.shape

(192, 3)

# finding best hyperparameters

In [34]:
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "random_state": 42,
        "tree_method": "hist",
    }

    model = xgb.XGBRegressor(**params)
    
    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
    )

    preds = model.predict(X_valid)
    return mean_squared_error(y_valid, preds)

In [35]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

# Print best results
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value:.4f}")
print("  Params:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-06-13 18:20:18,347] A new study created in memory with name: no-name-19751ce2-524e-4dfb-9b9f-83214572b98a


[0]	validation_0-rmse:54.66118
[1]	validation_0-rmse:52.85122
[2]	validation_0-rmse:51.38599
[3]	validation_0-rmse:49.69158
[4]	validation_0-rmse:48.06282
[5]	validation_0-rmse:46.76763
[6]	validation_0-rmse:45.50717
[7]	validation_0-rmse:43.95517
[8]	validation_0-rmse:42.48486
[9]	validation_0-rmse:41.36894
[10]	validation_0-rmse:40.30667
[11]	validation_0-rmse:39.25855
[12]	validation_0-rmse:37.94773
[13]	validation_0-rmse:37.01243
[14]	validation_0-rmse:35.82882
[15]	validation_0-rmse:34.94401
[16]	validation_0-rmse:33.77385
[17]	validation_0-rmse:33.03878
[18]	validation_0-rmse:31.97426
[19]	validation_0-rmse:30.92913
[20]	validation_0-rmse:29.95908
[21]	validation_0-rmse:29.03566
[22]	validation_0-rmse:28.35832
[23]	validation_0-rmse:27.76259
[24]	validation_0-rmse:26.93753
[25]	validation_0-rmse:26.36118
[26]	validation_0-rmse:25.58300
[27]	validation_0-rmse:24.86780
[28]	validation_0-rmse:24.09366
[29]	validation_0-rmse:23.42936
[30]	validation_0-rmse:22.74018
[31]	validation_0-

[I 2025-06-13 18:20:34,677] Trial 0 finished with value: 41.49518614063695 and parameters: {'n_estimators': 927, 'max_depth': 3, 'learning_rate': 0.03435570071888781, 'subsample': 0.9252561814262183, 'colsample_bytree': 0.6766686681767506, 'gamma': 3.295412358846876, 'reg_lambda': 0.09075522733572519, 'reg_alpha': 0.0014759103427466292}. Best is trial 0 with value: 41.49518614063695.


[0]	validation_0-rmse:47.76722
[1]	validation_0-rmse:40.75919
[2]	validation_0-rmse:36.18230
[3]	validation_0-rmse:30.99651
[4]	validation_0-rmse:26.92954
[5]	validation_0-rmse:24.06278
[6]	validation_0-rmse:21.42949
[7]	validation_0-rmse:18.91772
[8]	validation_0-rmse:16.88775
[9]	validation_0-rmse:15.55983
[10]	validation_0-rmse:14.29112
[11]	validation_0-rmse:13.37124
[12]	validation_0-rmse:12.17775
[13]	validation_0-rmse:11.45021
[14]	validation_0-rmse:10.78517
[15]	validation_0-rmse:10.34993
[16]	validation_0-rmse:9.66552
[17]	validation_0-rmse:9.58213
[18]	validation_0-rmse:8.93946
[19]	validation_0-rmse:8.39045
[20]	validation_0-rmse:7.79309
[21]	validation_0-rmse:7.28580
[22]	validation_0-rmse:7.00787
[23]	validation_0-rmse:6.83081
[24]	validation_0-rmse:6.60790
[25]	validation_0-rmse:6.34690
[26]	validation_0-rmse:6.16064
[27]	validation_0-rmse:5.93960
[28]	validation_0-rmse:5.72326
[29]	validation_0-rmse:5.61425
[30]	validation_0-rmse:5.48114
[31]	validation_0-rmse:5.34818
[3

[I 2025-06-13 18:20:50,360] Trial 1 finished with value: 15.744896859956036 and parameters: {'n_estimators': 877, 'max_depth': 15, 'learning_rate': 0.22918968472626156, 'subsample': 0.6419831229786132, 'colsample_bytree': 0.8470664929133345, 'gamma': 4.510427576141327, 'reg_lambda': 9.184491256942207, 'reg_alpha': 8.931281509663569e-08}. Best is trial 1 with value: 15.744896859956036.


...

[0]	validation_0-rmse:46.88866
[1]	validation_0-rmse:38.96250
[2]	validation_0-rmse:33.70850
[3]	validation_0-rmse:28.38428
[4]	validation_0-rmse:24.13509
[5]	validation_0-rmse:21.59914
[6]	validation_0-rmse:19.50153
[7]	validation_0-rmse:16.75142
[8]	validation_0-rmse:14.54076
[9]	validation_0-rmse:13.45606
[10]	validation_0-rmse:12.65942
[11]	validation_0-rmse:12.07899
[12]	validation_0-rmse:11.03997
[13]	validation_0-rmse:10.82136
[14]	validation_0-rmse:10.18109
[15]	validation_0-rmse:9.80708
[16]	validation_0-rmse:9.42387
[17]	validation_0-rmse:9.36868
[18]	validation_0-rmse:8.99711
[19]	validation_0-rmse:8.75149
[20]	validation_0-rmse:8.44195
[21]	validation_0-rmse:8.33363
[22]	validation_0-rmse:8.28377
[23]	validation_0-rmse:8.26534
[24]	validation_0-rmse:8.11173
[25]	validation_0-rmse:8.09258
[26]	validation_0-rmse:7.99541
[27]	validation_0-rmse:7.94682
[28]	validation_0-rmse:7.85575
[29]	validation_0-rmse:7.86070
[30]	validation_0-rmse:7.83963
[31]	validation_0-rmse:7.84432
[32

[I 2025-06-13 18:27:29,196] Trial 48 finished with value: 58.251536661518536 and parameters: {'n_estimators': 685, 'max_depth': 6, 'learning_rate': 0.19415320991649243, 'subsample': 0.9218547636876505, 'colsample_bytree': 0.7468554202977564, 'gamma': 4.423316032478621, 'reg_lambda': 1.3958074924364063, 'reg_alpha': 1.3753029065501334e-08}. Best is trial 36 with value: 13.351849633925037.


[0]	validation_0-rmse:52.65337
[1]	validation_0-rmse:47.50721
[2]	validation_0-rmse:45.38253
[3]	validation_0-rmse:43.37336
[4]	validation_0-rmse:41.98206
[5]	validation_0-rmse:38.75646
[6]	validation_0-rmse:36.87611
[7]	validation_0-rmse:35.01102
[8]	validation_0-rmse:31.86958
[9]	validation_0-rmse:30.92734
[10]	validation_0-rmse:30.31813
[11]	validation_0-rmse:29.88707
[12]	validation_0-rmse:27.48282
[13]	validation_0-rmse:26.84418
[14]	validation_0-rmse:24.91253
[15]	validation_0-rmse:24.36613
[16]	validation_0-rmse:24.15704
[17]	validation_0-rmse:23.98180
[18]	validation_0-rmse:22.48379
[19]	validation_0-rmse:22.37108
[20]	validation_0-rmse:21.28422
[21]	validation_0-rmse:20.41585
[22]	validation_0-rmse:20.26897
[23]	validation_0-rmse:20.02316
[24]	validation_0-rmse:19.44504
[25]	validation_0-rmse:19.45554
[26]	validation_0-rmse:19.23006
[27]	validation_0-rmse:19.39539
[28]	validation_0-rmse:18.85445
[29]	validation_0-rmse:18.89118
[30]	validation_0-rmse:18.63087
[31]	validation_0-

[I 2025-06-13 18:27:31,624] Trial 49 finished with value: 214.41777157696902 and parameters: {'n_estimators': 326, 'max_depth': 4, 'learning_rate': 0.11016891060008675, 'subsample': 0.5582724806441385, 'colsample_bytree': 0.6346969831022728, 'gamma': 4.776152159011557, 'reg_lambda': 0.3006410687769347, 'reg_alpha': 0.00024737068919638796}. Best is trial 36 with value: 13.351849633925037.


Best trial:
  Value: 13.3518
  Params:
    n_estimators: 530
    max_depth: 7
    learning_rate: 0.0626798085435809
    subsample: 0.622351462501602
    colsample_bytree: 0.7089588177139377
    gamma: 4.151407037242947
    reg_lambda: 6.355491781864828
    reg_alpha: 5.7908878727311974e-08


# Using Best Parameters

In [38]:
best_model = xgb.XGBRegressor(**study.best_params)
best_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.7089588177139377, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, feature_weights=None,
             gamma=4.151407037242947, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.0626798085435809,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=530, n_jobs=None,
             num_parallel_tree=None, ...)

# backtest

In [42]:
pred = best_model.predict(X_valid)

In [43]:
pred

array([180.58878, 172.65819, 144.99385, 162.18211, 157.64642, 207.95758,
       322.7133 , 140.16731, 176.51321, 186.36487, 161.32039, 293.28653,
       165.97488, 204.31718, 163.99765, 314.5738 , 214.81418, 180.63847,
       192.6133 , 172.1132 , 172.93318, 179.78952, 322.0967 , 180.66434,
       216.37543, 191.19475, 179.0297 , 263.99377, 311.49896, 196.2023 ,
       144.65663, 312.26172, 182.03638, 212.77618, 181.20833, 141.9411 ,
       236.38564, 146.25954, 140.01566, 212.2283 , 311.59244, 162.72841,
       301.38336, 184.34477, 196.87956, 143.2054 , 206.24234, 166.69997],
      dtype=float32)

In [45]:
error = mean_squared_error(y_valid, pred)
error

10.491329861028303

# Saving Best model

In [49]:
with open("best_model.pkl", "wb") as f:
    pkl.dump(best_model, f)